In [1]:
import random
import numpy as np

**Ex 1**

In [2]:
# Initialize a network
def initialize_network ( n_inputs , n_hidden , n_outputs ) :
  network = list ()
  hidden_layer = [{ 'weights':[ random.random() for i in range ( n_inputs + 1)]} for j in range ( n_hidden )]
  network.append(hidden_layer)
  output_layer = [{ 'weights':[ random.random() for i in range ( n_hidden + 1)]} for j in range ( n_outputs )]
  network.append(output_layer)
  return network


Les poids dans la liste hidden layers se situent entre le inputs layer et le hidden layer.<br>
Puis les poids dans la liste output lient la hidden layer et le output layer. Ces poids permettent de calculer la valeur de la couche output en faisant une comnbninaison linéaire. <br>
n_hidden represente le nombre de neurones des la hidden layer et non le nombre de hidden layer
<br>
<br>


**Ex 2**<br><br>
 Les deux premieres listes sont celles de hidden layer et les deux dernieres sont dans la layer output. Dans les listes du hidden layer, il y a les 3 poids des 3 inputs liés + le biais ce qui fait 4 valeurs. <br><br>
Et pour les deux listes du output layer, il y a 3 valeurs qui sont les 2 poids sortant des 2 neurones du hidden layer et le biais, qui est la troisième valeur.<br><br>
Le biais peut aussi être multiplié à un poids

In [3]:
random.seed(10)
neural_net = initialize_network(3,2,2)
for elm in neural_net:
  for sub in elm: 
    print(sub)

{'weights': [0.5714025946899135, 0.4288890546751146, 0.5780913011344704, 0.20609823213950174]}
{'weights': [0.81332125135732, 0.8235888725334455, 0.6534725339011758, 0.16022955651881965]}
{'weights': [0.5206693596399246, 0.32777281162209315, 0.24999667668640035]}
{'weights': [0.9528169091459117, 0.9965569925394552, 0.04455638245043303]}


<br><br>
**Ex 3**

La valeur d'activation est un entier qui résulte de l'application de la fonction y=x1*a+b*x2+... et on peut dire que si la valeur d'activation satisfait un certain seuil => le neurone sera activé, car la valeur qui ressort est un % => plus c'est proche de 1 plus la proba est haute


 <br> <br> <br> 
Le but de la DESCENTE DE GRADIENT est de réduire l'erreur (le biais/delta) pour  augmenter la valeur d'activation et démarrer le neurone. Pour cela les poids des inputs sont modifiés (BackPropagation)

<br> <br> <br>
Le nombre de input et nombre de neurone n'est pas obligé d'être le même, il peut y en avoir des différents.<br>
 On peut contribuer à ne pas activer un neurone en mettant un poids fortement négatif
 <br><br><br><br>
" En général au final on mets autant de neurones finaux qu'il y a de possibilités de labels (exemple des habitss MNIST) si on a un t-shirt, il faut que la valeur d'activation active le neurone lié aux tee shirt" car il aura un bon score
 <br>
 "Note : Quand on applique le log à une probabilité converge plus vite"
<br><br><br>
 La Fonction d'activation permet d'appliquer une fonction au resultat du noeud output qui est la somme des produit des inputs et des poids + le biais donc si on a 3 inputs un noeud neurone aura 3poids + 1 biais
 <br><br>

 On peut voir ce calcul comme la multiplication d'une matrice par un vecteur + le biais ce qui donnera en résultat le vecteur resultat de la couche output. 

 <br><br>


In [4]:

def activation(weights,inputs):
  res =0
  for i in range(len(weights)-1):
    res += weights[i]*inputs[i]
  res += weights[-1] # this represents the bias
  return res
  

<br><br><br><br>
Ex 4 

Il existe 3 principales fonctions qui habitent un neurone, cette fonction s'appelle FONCTION D'ACTIVATION. Avant que la valeur du neurone arrive dans le neurone output on lui applique la fonction d'activation qui va modifier la valeur en entrée du neurone. <br><br>
Il existe 3 principales fonctions d'activation : Sigmoid, tanh et reLu

In [5]:
#Sigmoid
def transfer(activation):
  return 1/(1+np.exp(-activation))

In [6]:
#tan h activation
def tanh(Z):
    A = np.tanh(Z)
    return A, Z

In [7]:
#relu activation
def relu(Z):
    A = np.maximum(0, Z)
    return A, Z

<br> <br> 
Ex 5

In [8]:
def forward_propagate ( network , row ):
  inputs = row
  for layer in network :
    new_inputs = []
    for neuron in layer :
      neuron['output'] = transfer(activation(neuron['weights'],inputs)) #inputs represente le valeur de la layer precedente
      new_inputs.append(neuron['output'])

    inputs = new_inputs
  return inputs


<br> <br>Ex 6 

The function returns the outputs from the last layer

In [9]:
random.seed(10)
row = [random.randint(0,1) for i in range(3)] # because 3= n_inputs input values 
print(forward_propagate(neural_net, row)) 

[0.7162125357710369, 0.833870768184967]


<br> <br><br><br>
Ex 7 Calculate the derivative of your transfer function

In [10]:
def transfer_derivative(output):
  return output*(1-output)
  

<br><br><br><br>Ex 8 Backpropagation : https://www.youtube.com/watch?v=0jh-jlWfKwo&ab_channel=HugoLarochelle   (7min29secondes) explication théorique on fait comme le forward propgation aux différences que :
<li> On part de la droite vers la gauche
<li> On ne prends plus la valeur du neurone mais son Delta qui est égal à <br> (= (1-Val_neurone_actuel) * Val_neurone_actuel *SOMME(poids_entre_neurone_actuel_et_neurone_precedent*delta_neurone_precedent)  : c'est le cas dans le 'else' dans le code ci-dessous<br>et simplement (y-val_neurone_actuel) si c'est la premiere occurence autrement dit que l'on commence au dernier output en forward prop avec y la valeur de la réelle du output final 


<br><br>
Le 'delta' est équivalent au coût, c'est ce qui caractérise l'erreur ressorti

In [11]:
#We reverse because in backprop we go from the output to the hidden layers
def backward_propagate_error(network, expected):
  for i in reversed(range(len(network))):
    layer = network[i]
    errors = list()

    if i == len(network)-1:
      for j in range(len(layer)):
        errors.append(expected[j] - layer[j]['output'])
    
    else:
      for j in range(len(layer)):
        error = 0.0
        for neuron in network[i+1]:
          error += (neuron['delta'] * neuron['weights'][j])
        errors.append(error)
    
    for j in range(len(layer)):
      neuron = layer[j]
      neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
    

<br><br> Ex 9 

In [12]:
backward_propagate_error(neural_net,[0.57,1.0])

In [13]:
neural_net

[[{'delta': 0.0011402027722892418,
   'output': 0.77084321637716,
   'weights': [0.5714025946899135,
    0.4288890546751146,
    0.5780913011344704,
    0.20609823213950174]},
  {'delta': 0.0017985861695399648,
   'output': 0.8371659811526281,
   'weights': [0.81332125135732,
    0.8235888725334455,
    0.6534725339011758,
    0.16022955651881965]}],
 [{'delta': -0.02971801069897396,
   'output': 0.7162125357710369,
   'weights': [0.5206693596399246, 0.32777281162209315, 0.24999667668640035]},
  {'delta': 0.023013934008580265,
   'output': 0.833870768184967,
   'weights': [0.9528169091459117, 0.9965569925394552, 0.04455638245043303]}]]

<br><br><br><br><br>

# TD 3

Ex 10

In [14]:
def update_weights(network, row, l_rate):
  inputs = row[:-1]
  for i in range(len(network)):
    new_inputs = []
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron['weights'][j]= neuron['weights'][j] + l_rate*neuron['delta']*inputs[j]
      neuron['weights'][-1] = l_rate*neuron['delta'] 
      new_inputs.append(neuron['output'])
    inputs = new_inputs


Ex 11

In [15]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs= forward_propagate(network, row)
      expected = [0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      print(expected)
      sum_error += np.mean(np.square([v1 - v2 for v1,v2 in zip(expected,outputs)]))
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    print(">epoch=%d, lrate =%.3f, error =%.3f" % (epoch , l_rate ,sum_error))

Ex 12 <br>
Randomly set values for data in order to find a training set

In [16]:
random.seed(11)
data = [[random.uniform(0,10), random.uniform(0,10), random.randint(0,1)] for i in range(10)]
data = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]


Ex 13 <br>

We take data as training set, 0.9 as learning rate, 20 as number of iteration and 2 as number of outputs

In [17]:
random.seed(13)
neural_network = initialize_network(2,2,2)
print(neural_network)
train_network(neural_network, data, 0.9, 20, 2)
print(neural_network)

[[{'weights': [0.2590084917154736, 0.6852579929645369, 0.6840819180161107]}, {'weights': [0.8493361613899302, 0.1857241738737354, 0.2305586089654681]}], [{'weights': [0.14715991816841778, 0.22516293556211264, 0.734023602212773]}, {'weights': [0.13021302275975688, 0.5313147518470183, 0.21390753049174072]}]]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
>epoch=0, lrate =0.900, error =2.428
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
>epoch=1, lrate =0.900, error =2.469
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
>epoch=2, lrate =0.900, error =2.479
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
>epoch=3, lrate =0.900, error =2.472
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
>epoch=4, lrate =0.900, error =2.428
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
>epoch=5, lrate =0.900, error =2.248
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[1, 0]
[0, 1]
[0, 1]
[

Ex 14

In [18]:
def predict(network, row):
  return np.argmax(forward_propagate(network,row))

In [19]:
for line in data:
  print("The real output is {} which is predicted as {}".format(line[-1],predict(neural_network,line)))

The real output is 0 which is predicted as 0
The real output is 0 which is predicted as 0
The real output is 0 which is predicted as 0
The real output is 0 which is predicted as 0
The real output is 0 which is predicted as 0
The real output is 1 which is predicted as 1
The real output is 1 which is predicted as 1
The real output is 1 which is predicted as 1
The real output is 1 which is predicted as 1
The real output is 1 which is predicted as 1


Ex 15-16

In [78]:
import csv


In [101]:

def load_csv(filename):
  data = []
  with open(filename,'r') as f :
    for l in f:
      l = l.replace('"','')
      tmp = l.split('\t')
      res = [float(x.strip()) for x in tmp]
      res[-1] = int(res[-1])
      data.append(res)
  
  return data


In [102]:
def load_csv(filecsv):
  df = list()
  x = list()
  value=''
  with open(filecsv,'r') as file:
    dataset = csv.reader(file) 
    for line in dataset:
      x = line[0].split('\t')
      x = list(map(eval,x))
      df.append(x)
  return df

In [103]:
seeds_dataset=load_csv('seeds_dataset.csv')
df=load_csv('seeds_dataset.csv')

<br><br><br><br>**Ex 17** :
<br> Shuffle,
<br> Split 80% to train and 20% to test on the dataset
<br> Find max and min for a col
<br>normalize columns except labels : To Normalize data we have to get the min and max. The formula for normalization is <br> x_norm= (x-x_min)/(x_max-x_min)

In [104]:
def find_extremums(dataset):
  return [[min(col),max(col)] for col in zip(*dataset)]

In [108]:
def normalize_data(dataset,extremum):
  for line in dataset :
    for element in range(len(line)-1) :
      line[element]=(line[element]-extremum[element][0])/(extremum[element][1]-extremum[element][0])
    line[-1] -=1
      

In [109]:
def normalize(dataset, extremums):
  # Normalize with minmax scaler and retrieve one from the label class to
  # have label between 0 and 2 instead of 1 and 3.
  for ind in dataset :
    for elm in range(len(ind)-1) :
      ind[elm]=(ind[elm]-extremums[elm][0])/(extremums[elm][1]-extremums[elm][0])
    ind[-1] -= 1
  
normalize(df, find_extremums(df))
random.shuffle(df)

In [110]:
seeds_dataset[:2]

[[0.4409820585457979,
  0.5020661157024793,
  0.570780399274047,
  0.48648648648648646,
  0.48610121168923714,
  0.18930164220052273,
  0.3451501723289019,
  1],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 1]]

In [111]:
extremum=find_extremums(seeds_dataset)
normalize_data(seeds_dataset,extremum)
seeds_dataset[:2]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0],
 [0.6962247673547146,
  0.8399802615346657,
  0.8929556781191885,
  0.8188579539251011,
  0.8026444954372717,
  0.10024538478746062,
  0.7431041774949315,
  0]]

In [112]:
random.shuffle(seeds_dataset)

In [113]:
def split_train_test_ratio(df,ratio):
  lim_test_train=int(len(df)*ratio)
  return df[:lim_test_train],df[lim_test_train:]

In [114]:
train,test=split_train_test_ratio(seeds_dataset,0.8)

In [115]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs= forward_propagate(network, row)
      expected = [0 for i in range(n_outputs)]
      expected[int(row[-1])] = 1
      sum_error += np.mean(np.square([v1 - v2 for v1,v2 in zip(expected,outputs)]))
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    print(">epoch=%d, lrate =%.3f, error =%.3f" % (epoch , l_rate ,sum_error))

<br><br><br><br><br>
Ex 18

In [116]:
neural_network = initialize_network(7, 5, 3)
train_network(neural_network, train, 0.9, 50, 3)

>epoch=0, lrate =0.900, error =43.342
>epoch=1, lrate =0.900, error =41.248
>epoch=2, lrate =0.900, error =40.312
>epoch=3, lrate =0.900, error =39.279
>epoch=4, lrate =0.900, error =37.840
>epoch=5, lrate =0.900, error =35.742
>epoch=6, lrate =0.900, error =32.933
>epoch=7, lrate =0.900, error =30.173
>epoch=8, lrate =0.900, error =28.311
>epoch=9, lrate =0.900, error =27.100
>epoch=10, lrate =0.900, error =26.148
>epoch=11, lrate =0.900, error =25.268
>epoch=12, lrate =0.900, error =24.365
>epoch=13, lrate =0.900, error =23.373
>epoch=14, lrate =0.900, error =22.241
>epoch=15, lrate =0.900, error =20.960
>epoch=16, lrate =0.900, error =19.599
>epoch=17, lrate =0.900, error =18.287
>epoch=18, lrate =0.900, error =17.132
>epoch=19, lrate =0.900, error =16.176
>epoch=20, lrate =0.900, error =15.406
>epoch=21, lrate =0.900, error =14.785
>epoch=22, lrate =0.900, error =14.270
>epoch=23, lrate =0.900, error =13.825
>epoch=24, lrate =0.900, error =13.426
>epoch=25, lrate =0.900, error =13.

Ex 19

In [117]:
def mse(y_true,y_pred):
  return np.square(np.subtract(y_true,y_pred)).mean()

def accuracy(neural_network, dataset):
  true=0
  for elm in dataset:
    if elm[-1] == predict(neural_network,elm) : true+=1
  return true/len(dataset)

In [118]:
accuracy(neural_network,test)

0.7857142857142857

In [119]:
#We add one to the both label to matching with the original class
for elm in test:
  print("{} and predicted as {}".format(elm[-1]+1,predict(neural_network,elm)+1))

2 and predicted as 2
1 and predicted as 2
3 and predicted as 3
3 and predicted as 3
2 and predicted as 2
3 and predicted as 3
2 and predicted as 2
3 and predicted as 3
3 and predicted as 3
1 and predicted as 1
1 and predicted as 1
1 and predicted as 2
1 and predicted as 1
1 and predicted as 1
1 and predicted as 1
2 and predicted as 2
1 and predicted as 1
3 and predicted as 1
1 and predicted as 3
3 and predicted as 3
1 and predicted as 2
2 and predicted as 2
1 and predicted as 3
2 and predicted as 2
1 and predicted as 2
3 and predicted as 3
2 and predicted as 1
2 and predicted as 2
1 and predicted as 3
1 and predicted as 1
3 and predicted as 3
1 and predicted as 1
3 and predicted as 3
3 and predicted as 3
3 and predicted as 3
2 and predicted as 2
3 and predicted as 3
2 and predicted as 2
2 and predicted as 2
1 and predicted as 1
3 and predicted as 3
1 and predicted as 1
